# Selecting relevant features in a linear relation

Let
$d_x$, $d_y$, $d_z$
be positive integers.
Let $X$
be a $d_x$-dimensional random vector.
Let 
$$
T: \mathbb{R}^{d_x} \rightarrow \mathbb{R}^{d_y}
$$
be a linear map.
Let 
$
Y = TX
$,
and let
$
\sigma(Y)
$
be the $\sigma$-algebra generated by $Y$. 



For linear transformations
$$
P: \mathbb{R}^{d_x} \rightarrow \mathbb{R}^{d_z},
$$
let $\sigma(PX)$ be the $\sigma$-algebra generated by $Z=PX$.
We call $Z=PX$ the projection of $X$ via $P$. 


We are interested in finding a projection of $X$ such that $\sigma(Y) \subset \sigma(PX)$.
This is not always possible. 
However,
we asume that such projecton exists,
and we run our feature-selecxtion algorithm.

Notice that
if $P$ is a $d_z \times d_x$ real matrix
such that $\sigma(Y) \subset \sigma(PX)$, 
then
for any invertible $d_z \times d_z$ real matrix $A$
and any invertible $d_x \times d_x$ real matrix $B$,
the matrix $APB$ is also such that $\sigma(Y) \subset \sigma(APBX)$.


Hence,
we consider the minimisation problem

$$
\inf \left\lbrace
\sigma(Y) \setminus \sigma(PX):
\,\, 
P \in \text{Hom}\left(\mathbb{R}^{d_x}, \mathbb{R}^{d_z}\right)
\right\rbrace,
$$
where $P$ ranges in the space of homomorphisms from $\mathbb{R}^{d_z}$ to $\mathbb{R}^{d_z}$.
 


We represent an approximate minimiser for this minimisation problem by using HSIC Lasso. 

In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import special_ortho_group

from hisel.select import Selector

## Experiment 1: Choose first coordinate of two

In Experiment 1, we take
$d_x = 2$,
$d_y = 1$,
$d_z = 1$.
Moreover, we let $X$ be uniformly distributed in the unit sqaure $[0, 1] \times [0, 1]$, and we consider the transformation defined by the $1\times 2$ real matrix
$$
T = 
\left(
1, 
\quad
0
\right).
$$

After the gradient descent has stopped, 
the training is successful if 
$P$ is such that
$$
P = \left(
a,
\quad
0
\right)
$$
where $\lvert a \rvert  > 0$. 

Below, we will test the fitness of $P$
by relying on linear regression scores. 

In [ ]:
transform = np.array([[1., 0.]], dtype=float)
dim_x = 2
dim_y = 1
dim_z = 1
batch_size = int(1e+4)
minibatch_size = int(4e2)
num_of_samples = int(1e+5)
number_of_epochs = 2

In [ ]:
x_samples = np.random.uniform(size=(num_of_samples, dim_x))
tt = np.repeat(np.expand_dims(transform, axis=0), repeats=num_of_samples, axis=0)
y_samples = (tt @ np.expand_dims(x_samples, axis=2))[:, :, 0]

In [ ]:
projector = Selector(x_samples, y_samples)

#### Fit to the data

In [ ]:
t0 = datetime.datetime.now()
p = projector.projection_matrix(
    number_of_features=dim_z,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
)
t1 = datetime.datetime.now()

In [ ]:
dt = t1 - t0
print(f'Training time: {dt}')

#### Trained $P$

In [ ]:
print(f'Final value of P:\n{p}')

## Experiment 2: 50D input, 5D projection, 3D target

In Experiment 2, we take
$d_x = 50$,
$d_y = 3$,
$d_z = 5$.
Moreover, we let $X$ be uniformly distributed in the unit cube $[0, 1]^{d_x}$, and we consider the transformation defined by the $3 \times 50$ real matrix
$$
T = 
\left(
\tilde{T}
\quad
\vert
\quad
0
\right),
$$
where 
$$
\tilde{T} 
=
\left( I \,\, \vert \,\, 0 \right)
\cdot
S,
$$
where
$I$ is the $d_y \times d_y$ identify matrix,
$0$ is the $d_y \times (d_z - d_y)$ null matrix,
and $S$ is sampled from 
the Haar distribution on the special orthgoonal group $O(d_z)$.



The selection is successful 
if $P$ is such that 
$$
P = \left(
\tilde{P},
\quad
0
\right)
$$
where $0$ is the zero matrix of dimension $d_z \times (d_x - d_z)$,
and $\text{rank}(\tilde{P})\geq 3$. 

In [ ]:
dim_x = 50
dim_y = 3
dim_z = 5
transform = np.concatenate((special_ortho_group.rvs(dim_z)[:dim_y], np.zeros((dim_y, dim_x - dim_z))), axis=1)
batch_size = int(1e+4)
minibatch_size = 300
num_of_samples = int(1e+4)
number_of_epochs = 3

In [ ]:
x_samples = np.random.uniform(size=(num_of_samples, dim_x))
tt = np.repeat(np.expand_dims(transform, axis=0), repeats=num_of_samples, axis=0)
y_samples = (tt @ np.expand_dims(x_samples, axis=2))[:, :, 0]

In [ ]:
projector = Selector(x_samples, y_samples)

#### Fit to the data

In [ ]:
t0 = datetime.datetime.now()
p = projector.projection_matrix(
    number_of_features=dim_z,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
)
t1 = datetime.datetime.now()

In [ ]:
dt = t1 - t0
print(f'Training time: {dt}')

#### Visualisation of trained $\lvert P \rvert$

In [ ]:
ax = sns.heatmap(np.abs(p), annot=False, linewidths=.5, square=True, cmap='Reds')
ax.set_title('Absolute value of projection matrix after training')

## Experiment 3: 100D input, 2D target, unknown `dim_z`, and random shuffle of features

In Experiment 3, we take
$d_x = 100$,
$d_y = 2$,
$d_z = 20$.
We let $X$ be uniformly distributed in the unit cube $[0, 1]^{d_x}$.
Moreover, 
let $d_t$ be a random integer in the interval $18 \leq d_t\leq 22$.
and let 
$S$ be sampled from the Haar distribution 
on the special orthogonal group $O(d_t)$.
Let
$$
\tilde{T} 
=
\left( I \,\, \vert \,\, 0 \right)
\cdot
S,
$$
where
$I$ is the $d_y \times d_y$ identify matrix,
$0$ is the $d_y \times (d_t - d_y)$ null matrix.
Let $A$ be the $d_t \times d_x$ matrix obtained by randomly permuting the columns of 
$$
\left( I \,\, \vert \,\, 0 \right)  \, \, \in {M}_{d_t\times d_x}(\mathbb{R}).
$$
We let $T$ be defined by
$$
T = \tilde{T}A.
$$

In [ ]:
dim_x = 100
dim_y = 2
dim_z = 20
dim_t = np.random.randint(low=18, high=23)
transform_tilde = special_ortho_group.rvs(dim_t)[:dim_y]
A = np.random.permutation(np.concatenate((np.eye(dim_t), np.zeros((dim_t, dim_x - dim_t))), axis=1).T).T
transform = transform_tilde @ A

batch_size = int(1e+4)
minibatch_size = 250
num_of_samples = int(2e+4)
number_of_epochs = 3

In [ ]:
print(f'Dimensionality of the transformation, dim_t: {dim_t}')
print(f'Dimensionality of the projection, dim_z: {dim_z}')

In [ ]:
x_samples = np.random.uniform(size=(num_of_samples, dim_x))
tt = np.repeat(np.expand_dims(transform, axis=0), repeats=num_of_samples, axis=0)
y_samples = (tt @ np.expand_dims(x_samples, axis=2))[:, :, 0]

In [ ]:
projector = Selector(x_samples, y_samples)

#### Fit to the data

In [ ]:
t0 = datetime.datetime.now()
p = projector.projection_matrix(
    number_of_features=dim_z,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
)
t1 = datetime.datetime.now()

In [ ]:
dt = t1 - t0
print(f'Training time: {dt}')

#### Visualisation of trained $\lvert P \rvert$

In [ ]:
ax = sns.heatmap(np.abs(p), annot=False, linewidths=.5, square=True, cmap='Reds')
ax.set_title('Absolute value of projection matrix after training')

### Reconciliation with $A$ and $T$

We keep only the entries in the trained $P$ that are significantly not zero. 

On the one hand, 
we look at the columns 
with significantly non-zero entries,
and we compare these columns with the columns of $A$. 
A perfectly trained $P$ would have 
significantly non-zero entries only in columns 
that correspond to non-null columns of $A$. 

On the other hand,
we look at the null columns of $P$,
and we compare these columns with the columns of $T$.
A perfectly trained $P$ would 
have 
null columns 
that correspond to null columns of $T$. 

In [ ]:
a_recon = np.argsort(np.linalg.norm(p, axis=0))[::-1]
t_recon = np.linalg.norm(p, axis=0) < 1e-3

$P$ reconciles with $A$ if the following list of zeros and ones is sorted in decreasing order.

In [ ]:
print(f'norm of columns of A arranged '
      'in the decreasing oder of norms of columns of P:\n'
      f'{np.linalg.norm(A, axis=0)[a_recon]}')

$P$ reconciles with $T$ if all entries of the following array are zero, except for possibly $d_t - d_z$ entries, if this difference is positive.

In [ ]:
print(f'norm of columns of T '
      'corresponding to null columns of P:\n'
      f'{np.linalg.norm(transform, axis=0)[t_recon]}\n')
print(f'number of expected non-null entries: {max(0, dim_t - dim_z)}\n')